In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 t-SNE 繪製從文字文件進行嵌入相似度視覺化

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/vector-search/embedding-similarity-visualization.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory 標誌"><br> 在 Colab 中執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/vector-search/embedding-similarity-visualization.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub 標誌"><br> 在 GitHub 上檢視
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/vector-search/embedding-similarity-visualization.zh.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="頂點 AI 標誌"><br> 在頂點 AI 工作空間中開啟
    </a>
  </td>
</table>


| | |
|-|-|
|作者(人) | [Gabe Rives-Corbett](https://github.com/grivescorbett) |


此筆記本示範向量相似性如何與 LLM 生成的嵌入式有關。你會將一組標籤式文件嵌入，然後將嵌入式繪製在二維的 t-SNE 繪圖中，以根據嵌入式來觀察相似文件是如何傾向於聚集在一起的。


## 開始使用


### 安裝函式庫


In [ ]:
!pip install --user langchain==0.0.315 \
                    google-cloud-aiplatform==1.35.0 \
                    scikit-learn==1.3.1

### 重新啟動目前的執行階段

要在此 Jupyter 執行階段中使用新安裝的套件，你必須重新啟動執行階段。你可以執行下列Cell來執行此項操作，如此將重新啟動目前的Kernel。


In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython
import time

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ Kernel將重新啟動。請等待它完成，再繼續執行下一個步驟。⚠️</b>
</div>


### 認證你的 notebook 環境 (僅限 Colab) 

如果你是在 Google Colab 上執行這個筆記本，你將需要認證你的環境。為執行這項工作，請執行下列新的Cell。如果你使用的是 [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench)，則不需要執行這個步驟。


In [ ]:
import sys

if "google.colab" in sys.modules:
    # Define project information
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
    LOCATION = "us-central1"  # @param {type:"string"}

    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### 匯入函式庫


In [ ]:
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from google.api_core import retry
from sklearn.datasets import fetch_20newsgroups
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
from vertexai.language_models import TextEmbeddingModel

tqdm.pandas()

## 取得並清理數據

在這個範例中，你將使用公開來源 [20 Newsgroups](http://qwone.com/~jason/20Newsgroups/) 資料集，此資料集是約 20,000 個新聞群組文件，平均分為 20 個不同的新聞群組


In [ ]:
categories = ["comp.graphics", "sci.space", "sci.med", "rec.sport.hockey"]
newsgroups = fetch_20newsgroups(categories=categories)

In [ ]:
raw_data = pd.DataFrame()
raw_data["text"] = newsgroups.data
raw_data["target"] = [newsgroups.target_names[x] for x in newsgroups.target]

由於 8k 輸入 token 限制，在此範例中，你將排除長度超出此限制的所有文件。

即使 token 通常 >=1 個字元，為了簡潔起見，你只要過濾小於或等於 8000 個 _字元_ 的文件即可。


In [ ]:
filtered = raw_data.loc[raw_data["text"].str.len() <= 8000]

對資料集進行分層標籤，採樣 500 個資料點


In [ ]:
x_subsample, _, y_subsample, _ = train_test_split(
    raw_data["text"], raw_data["target"], stratify=raw_data["target"], train_size=500
)

透過移除電子郵件、人名等來清除文本。這將有助於改善將被轉換成嵌入碼的數據。


In [ ]:
x_subsample = [re.sub(r"[\w\.-]+@[\w\.-]+", "", d) for d in x_subsample]  # Remove email
x_subsample = [re.sub(r"\([^()]*\)", "", d) for d in x_subsample]  # Remove names
x_subsample = [d.replace("From: ", "") for d in x_subsample]  # Remove "From: "
x_subsample = [
    d.replace("\nSubject: ", "") for d in x_subsample
]  # Remove "\nSubject: "

In [ ]:
df = pd.DataFrame()
df["text"] = x_subsample
df["target"] = list(y_subsample)

你現在有 500 個資料點，大致均勻分佈在各類別中：


In [ ]:
df["target"].value_counts()

## 使用 t-SNE 圖形建立並視覺化嵌入

從 Vertex AI 載入文字嵌入模型 ([說明文件](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/text-embeddings))。


In [ ]:
model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

In [ ]:
# Retrieve embeddings from the specified model with retry logic
def make_embed_text_fn(model):
    @retry.Retry(timeout=300.0)
    def embed_fn(text):
        return model.get_embeddings([text])[0].values

    return embed_fn

建立嵌入式處理。這可能會花費一、兩分鐘。


In [ ]:
df["embeddings"] = df["text"].progress_apply(make_embed_text_fn(model))

In [ ]:
df.head()

我們模型產生的向量有 768 個維度，因此無法在 768 個維度上進行視覺化。你可以使用 [t-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding) 將其壓縮成 2 個維度。


In [ ]:
embeddings_array = np.array(df["embeddings"].to_list(), dtype=np.float32)
tsne = TSNE(random_state=0, n_iter=1000)
tsne_results = tsne.fit_transform(embeddings_array)

In [ ]:
df_tsne = pd.DataFrame(tsne_results, columns=["TSNE1", "TSNE2"])
df_tsne["target"] = df["target"]  # Add labels column from df_train to df_tsne

In [ ]:
df_tsne.head()

將資料點繪製出來。現在在使用文字嵌入後，應可直觀地看出，來自相同新聞群組的文件在向量空間中顯示在彼此的附近。


In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))  # Set figsize
sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})
sns.scatterplot(data=df_tsne, x="TSNE1", y="TSNE2", hue="target", palette="hls")
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.title("Scatter plot of news using t-SNE")
plt.xlabel("TSNE1")
plt.ylabel("TSNE2")
plt.axis("equal")